In [24]:
import pandas as pd
import os
from termcolor import colored
run_setting_f=open("run_setting_miseq.csv")
run_config_dict={}
sample_dict={}
df=pd.DataFrame(columns=['sample_id','concentration','pct','pct/con'])

for line in run_setting_f:
    if line.startswith("sample ID"):
        break
    items=line.split(",")
    run_config_dict[items[0]]=items[1]
    
for line in run_setting_f:
    items=line.strip().split(",")
    print (items)
    if items[1]=='':
        df=df.append({'sample_id':items[0],'concentration':items[1],'pct':float(items[2])},ignore_index=True)
    else:
        df=df.append({'sample_id':items[0],'concentration':float(items[1]),'pct':float(items[2]),
                                                             'pct/con':float(items[2])/float(items[1])},ignore_index=True)
    
file_name='%s_Miseq_%s_%s+%s_%s.xlsx' %(run_config_dict['date'],run_config_dict['nano/micro/v3'],run_config_dict['R1'],
                                   run_config_dict['R2'],run_config_dict['project_id'])


#final loading volume
size=int(run_config_dict['size of library'])
target_cluster_density=int(run_config_dict['target_cluster_density'])

if size<=300:
    loading_pM=target_cluster_density/125
    phix_cluster_efficiency=2.1
elif size<=400:
    loading_pM=target_cluster_density/115
    phix_cluster_efficiency=3
elif size<=500:
    loading_pM=target_cluster_density/105
    phix_cluster_efficiency=3.2
elif size>500:
    loading_pM=target_cluster_density/95
    phix_cluster_efficiency=3.2
    
    

final_loading_20pM_sample=600*float(run_config_dict['% of library'])*loading_pM/20
final_loading_20pM_phix=target_cluster_density*float(run_config_dict['% of phix'])/phix_cluster_efficiency
HT1=600-final_loading_20pM_sample-final_loading_20pM_phix

#to check if the total pct over 100
if df['pct'].sum() >102 or df['pct'].sum()<95:
    print (colored('pencentage is wrong','red'))
    

run_setting_f.close()    


['18034-2301-01(WEP22-491)', '39.75', '100']


In [26]:
#if there is only one sample and only do not have qPCR result yet
from openpyxl import load_workbook

if len(df)==1 and df['concentration'][0]=='':
    wb_loading=load_workbook("./Miseq_loading_template.xlsx")
    ws_loading=wb_loading['loading']
    title="Miseq_%s_%s+%s+%s+%s" %(run_config_dict['nano/micro/v3'],run_config_dict['R1'],run_config_dict['I1'],
                              run_config_dict['I2'],run_config_dict['R2'])
    ws_loading.cell(row=1,column=1).value=title
    ws_loading.cell(row=4,column=2).value=df['sample_id'][0]
    ws_loading.cell(row=4,column=4).value='=IF(C4>20,2,2*10/C4)'
    ws_loading.cell(row=4,column=5).value='=D4*C4/2-D4'
    ws_loading.cell(row=4,column=6).value='10ul 2nM sample+10ul 0.1 NaOH+980ul HT1'
    ws_loading.cell(row=4,column=7).value=final_loading_20pM_sample
    ws_loading.cell(row=4,column=8).value=final_loading_20pM_phix
    ws_loading.cell(row=4,column=9).value=HT1
    
    row_now=5
    #customer primer 
    if run_config_dict['customer/spike-in']== '':
        pass
    else:
        read1_primer=run_config_dict['read1 primer']
        read2_primer=run_config_dict['read2 primer']
        index1_primer=run_config_dict['index1 primer']

        if run_config_dict['customer/spike-in']=='spike-in':
            if read1_primer !='':
                read1_primer_add='3.4ul 100uM Read1 primer into #12'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read1_primer
                ws_loading.cell(row=row_now,column=6).value=read1_primer_add            
            if index1_primer !='':
                index1_primer_add='3.4ul 100uM index1 primer into #13'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=index1_primer
                ws_loading.cell(row=row_now,column=6).value=index1_primer_add          
            if read2_primer !='':
                read2_primer_add='3.4ul 100uM read2 primer into #14'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read2_primer
                ws_loading.cell(row=row_now,column=6).value=read2_primer_add

        elif run_config_dict['customer/spike-in']=='customer':
            if read1_primer !='':
                read1_primer_add='3ul 100uM read1 primer+597ul HT1 -->#18'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read1_primer
                ws_loading.cell(row=row_now,column=6).value=read1_primer_add
            if index1_primer !='':   
                index1_primer_add='3ul 100uM index primer+597ul HT1 -->#19'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=index1_primer
                ws_loading.cell(row=row_now,column=6).value=index1_primer_add              
            if read2_primer !='':   
                read2_primer_add='3ul 100uM read2 primer+597ul HT1 -->#20'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read2_primer
                ws_loading.cell(row=row_now,column=6).value=read2_primer_add            


            row_now+=1
            ws_loading.cell(row=row_now,column=5).value='need to change sample sheet'
        elif run_config_dict['customer/spike-in'] is not None:
            print (colored("customer primer is not writing correctly","red"))


    row_now+=2
    ws_loading.cell(row=row_now,column=5).value='Box'
    ws_loading.cell(row=row_now,column=6).value=run_config_dict['sample pos']
    wb_loading.save(file_name)
    os.startfile(file_name)

In [27]:
#divied the sample into different pools
dict_of_df={}

low_pct_con=min(df['pct/con'])
for i in range(1,10):
    if len(df)==0:
        break
    #based on pct/con value, divided into different pool 
    dict_of_df['df_pool%s' %i]=[df[df['pct/con']<=low_pct_con*10].reset_index(drop=True),0]
    #the rest sample will be in another pool
    df=df[df['pct/con']>low_pct_con*10]
    try:
        low_pct_con=min(df['pct/con'])
    except:
        break

for df_name,df_itself_and_water in dict_of_df.items():
    if len(df_itself_and_water[0])>1:
        low_pct_con=10000000000
        total_mol=0
        for i,j in df_itself_and_water[0].sort_values(by='pct/con',ascending=True).iterrows():
            print (i)
            low_pct_con=min(low_pct_con,j['pct/con'])
            df_itself_and_water[0].at[i,'add vol']=round(j['pct/con']/low_pct_con,2)
            total_mol+=j['pct/con']/low_pct_con*j['concentration']
            print ('add value = %s' %(j['pct/con']/low_pct_con) )

        #water volumn addded
        water_vol=total_mol/2-df_itself_and_water[0]['add vol'].sum()
        dict_of_df[df_name][1]=water_vol
        print (water_vol)

    elif len(df_itself_and_water[0])==1:
        if df_itself_and_water[0]['concentration'][0] > 20:
            df_itself_and_water[0].at[0,'add vol']=2
            water_vol=2*df_itself_and_water[0]['concentration'][0]/2-2
        else:
            df_itself_and_water[0].at[0,'add vol']=20/df_itself_and_water[0]['concentration'][0]
            water_vol=10-20/df_itself_and_water[0]['concentration'][0]
        dict_of_df[df_name][1]=water_vol
     

In [28]:
#generate excel file
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Alignment
from openpyxl import Workbook
from openpyxl import load_workbook

from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter

wb_loading=load_workbook("./Miseq_loading_template.xlsx")
ws_loading=wb_loading['loading']

thin = Side(border_style="thin", color="000000")

title="Miseq_%s_%s+%s+%s+%s" %(run_config_dict['nano/micro/v3'],run_config_dict['R1'],run_config_dict['I1'],
                              run_config_dict['I2'],run_config_dict['R2'])
ws_loading.cell(row=1,column=1).value=title

row_start=4
row_now=3
for df_name,df_itself_and_water in dict_of_df.items():
    df_itself=df_itself_and_water[0]
    pool_id=run_config_dict['project_id']+'-loading-'+df_name.split('_')[1]
    ws_loading.cell(row=row_start,column=1).value=pool_id
    
    for index,sample in df_itself.iterrows():
        row_now+=1   
        ws_loading.cell(row=row_now,column=2).value=sample['sample_id']
        ws_loading.cell(row=row_now,column=3).value=sample['concentration']
        ws_loading.cell(row=row_now,column=4).value=sample['add vol']
    
    ws_loading.cell(row=row_start,column=5).value=df_itself_and_water[1]
    ws_loading.cell(row=row_start,column=6).value='10ul 2nM sample+10ul 0.1 NaOH+980ul HT1'
    ws_loading.cell(row=row_start,column=7).value=final_loading_20pM_sample*df_itself['pct'].sum()/100
    #merge the cell for different pool
    for col,col_num in zip(['A','E','F','G'],[1,5,6,7]):
        print (col,col_num)
        ws_loading.merge_cells("%s%s:%s%s" %(col,row_start,col,row_now))
        ws_loading.cell(row=row_start,column=col_num).alignment=Alignment(horizontal='center',vertical='center')
    
    row_now+=1
    row_start=row_now+1

#merge the cell for final loading part

ws_loading.cell(row=4,column=8).value=final_loading_20pM_phix
ws_loading.cell(row=4,column=9).value=HT1

for col,col_num in zip(['H','I'],range(8,10)):
    print (col,col_num)
    ws_loading.merge_cells("%s4:%s%s" %(col,col,row_now-1))
    ws_loading.cell(row=4,column=col_num).alignment=Alignment(horizontal='center',vertical='center')

for row in ws_loading['A%s:I%s' %(1,row_now-1)]:
    for cell in row:
        cell.border=Border(top=thin,left=thin,right=thin,bottom=thin)
        

row_now+=1
#customer primer 
if run_config_dict['customer/spike-in']== '':
    pass
else:
    read1_primer=run_config_dict['read1 primer']
    read2_primer=run_config_dict['read2 primer']
    index1_primer=run_config_dict['index1 primer']

    if run_config_dict['customer/spike-in']=='spike-in':
        if read1_primer !='':
            read1_primer_add='3.4ul 100uM Read1 primer into #12'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read1_primer
            ws_loading.cell(row=row_now,column=6).value=read1_primer_add            
        if index1_primer !='':
            index1_primer_add='3.4ul 100uM index1 primer into #13'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=index1_primer
            ws_loading.cell(row=row_now,column=6).value=index1_primer_add          
        if read2_primer !='':
            read2_primer_add='3.4ul 100uM read2 primer into #14'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read2_primer
            ws_loading.cell(row=row_now,column=6).value=read2_primer_add
        
    elif run_config_dict['customer/spike-in']=='customer':
        if read1_primer !='':
            read1_primer_add='3ul 100uM read1 primer+597ul HT1 -->#18'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read1_primer
            ws_loading.cell(row=row_now,column=6).value=read1_primer_add
        if index1_primer !='':   
            index1_primer_add='3ul 100uM index primer+597ul HT1 -->#19'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=index1_primer
            ws_loading.cell(row=row_now,column=6).value=index1_primer_add              
        if read2_primer !='':   
            read2_primer_add='3ul 100uM read2 primer+597ul HT1 -->#20'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read2_primer
            ws_loading.cell(row=row_now,column=6).value=read2_primer_add            
        
        
        row_now+=1
        ws_loading.cell(row=row_now,column=5).value='need to change sample sheet'
    elif run_config_dict['customer/spike-in'] is not None:
        print (colored("customer primer is not writing correctly","red"))


row_now+=2
ws_loading.cell(row=row_now,column=5).value='Box'
ws_loading.cell(row=row_now,column=6).value=run_config_dict['sample pos']



wb_loading.save(file_name)
os.startfile(file_name)


A 1
E 5
F 6
G 7
H 8
I 9


In [29]:
#generate sample sheet template
sample_sheet_template_f=open('./SampleSheet_miseq_template.csv')
output_sample_sheet=open('%s_%s_SampleSheet.csv' %(run_config_dict['date'],run_config_dict['project_id']),'w+')

for index,line in enumerate(sample_sheet_template_f):
    if index==3:
        output_sample_sheet.write("Experiment-Name,%s\n" %run_config_dict['project_id'] )
    elif index==4:
        output_sample_sheet.write("Date,%s\n" %run_config_dict['date'])
    elif index==12:
        output_sample_sheet.write('%s\n' %(run_config_dict['R1']))
    elif index==13:
        if run_config_dict['R2'] !='0':
            output_sample_sheet.write("%s\n" %run_config_dict['R2'] )
    else:
        output_sample_sheet.write(line)
sample_sheet_template_f.close()
output_sample_sheet.close()

os.startfile('%s_%s_SampleSheet.csv' %(run_config_dict['date'],run_config_dict['project_id']))
    
